# Session 3: Modelo v1 - Conectividad de Corredores Ecológicos

**Fecha:** 29 de octubre de 2025  
**Versión:** v1_connectivity_milp  
**Estado:** 🚀 En desarrollo  

Evolución natural del modelo v0 (Greedy baseline) incorporando **conectividad ecológica** mediante corredores entre celdas adyacentes.

## Objetivo

Maximizar: Cobertura ponderada de hábitats + Conectividad efectiva de corredores  
Sujeto a: Presupuesto limitado, restricciones de adyacencia, validez geométrica

## Cambios respecto a v0

| Aspecto | v0 (Greedy) | v1 (MILP + Conectividad) |
|---------|-------------|-------------------------|
| **Solver** | Heurístico (Python) | MILP exacto (HiGHS/CBC) |
| **Variables** | x[i,s] adaptación | x[i,s] + y[i,j,s] corredores |
| **Función objetivo** | Cobertura ponderada | Cobertura + λ·Conectividad |
| **Corredores** | No | Sí, con coste |
| **Tiempo** | < 1s | Minutos (según tamaño) |
| **Optimalidad** | Aproximada | Exacta |

---

## Sección 1: Importar Librerías y Cargar Datos

In [ ]:
import sysimport osimport jsonimport warningsfrom datetime import datetimefrom pathlib import Pathimport time# Configurar paths absolutos PRIMEROBASE_PATH = Path('/home/ayuda137/Escritorio/asuntos/ internos/menorca-optimization')DATA_PATH = BASE_PATH / 'data'NOTEBOOKS_PATH = BASE_PATH / 'notebooks'os.chdir(str(BASE_PATH))  # Cambiar directorio de trabajo# Resto de imports...import pandas as pdimport numpy as npimport geopandas as gpdfrom shapely.geometry import Point, LineStringfrom shapely.strtree import STRtreeimport matplotlib.pyplot as pltimport matplotlib.patches as mpatchesfrom matplotlib.collections import LineCollectionfrom matplotlib.colors import LinearSegmentedColormapfrom pyomo.environ import (    ConcreteModel, Param, Set, Var, Objective, Constraint,    Binary, maximize, SolverFactory, TerminationCondition, SolverStatus,    value)warnings.filterwarnings('ignore')print("✓ Librerías importadas correctamente")print(f"✓ Python versión: {sys.version.split()[0]}")print(f"✓ Directorio de trabajo: {os.getcwd()}")

In [ ]:
# Cargar dataset procesado
dataset_path = DATA_PATH / 'dataset_processed.geojson'

try:
    gdf = gpd.read_file(dataset_path)
    print(f"✓ Dataset cargado: {len(gdf)} celdas")
    print(f"✓ Columnas: {len(gdf.columns)}")
    print(f"✓ CRS: {gdf.crs}")
    print(f"✓ Geometría válida: {gdf.geometry.is_valid.sum()}/{len(gdf)}")
except Exception as e:
    print(f"✗ Error al cargar dataset: {e}")
    raise

# Cargar configuración v0
config_v0_path = DATA_PATH / 'model_config_v0.json'
try:
    with open(config_v0_path, 'r') as f:
        config_v0 = json.load(f)
    print(f"✓ Configuración v0 cargada")
except Exception as e:
    print(f"⚠ Archivo de configuración v0 no encontrado: {e}")
    config_v0 = {}

## Sección 2: Construir Grafo de Adyacencia Espacial

In [ ]:
print("Construyendo matriz de adyacencia espacial...")
print(f"Geometrías: {len(gdf)}")

# Usar STRtree para encontrar vecinos de forma eficiente
tree = STRtree(gdf.geometry)

# Generar pares de celdas adyacentes (que se tocan)
edges = []
edge_dict = {}  # Para evitar duplicados

for idx, row in gdf.iterrows():
    grid_id_i = row['grid_id']
    geom_i = row['geometry']
    
    # Encontrar vecinos (que comparten borde o vértice)
    neighbors_idx = tree.query(geom_i.buffer(0.001), predicate='touches')
    
    for neighbor_idx in neighbors_idx:
        if neighbor_idx > idx:  # Evitar duplicados
            grid_id_j = gdf.iloc[neighbor_idx]['grid_id']
            # Coste uniforme del corredor (normalizado)
            cost_corridor = 0.1  # Valor bajo para incentivar conectividad
            edges.append({
                'cell_i': grid_id_i,
                'cell_j': grid_id_j,
                'cost_corridor': cost_corridor
            })
            edge_dict[(grid_id_i, grid_id_j)] = cost_corridor

print(f"✓ Adyacencias encontradas: {len(edges)}")

# Guardar en CSV
edges_df = pd.DataFrame(edges)
edges_path = DATA_PATH / 'corridor_adjacency.csv'
edges_df.to_csv(edges_path, index=False)
print(f"✓ Matriz de adyacencia guardada: {edges_path}")

# Estadísticas
print(f"\nEstadísticas de adyacencia:")
print(f"  Pares de celdas vecinas: {len(edges)}")
print(f"  Coste mínimo de corredor: {min(e['cost_corridor'] for e in edges):.3f}")
print(f"  Coste máximo de corredor: {max(e['cost_corridor'] for e in edges):.3f}")

## Sección 3: Preparar Parámetros del Modelo

In [ ]:
# Definir especies
SPECIES = {
    'atelerix': 'has_atelerix_algirus',
    'martes': 'has_martes_martes',
    'eliomys': 'has_eliomys_quercinus',
    'oryctolagus': 'has_oryctolagus_cuniculus'
}

COST_COLS = {
    'atelerix': 'cost_adaptation_atelerix',
    'martes': 'cost_adaptation_martes',
    'eliomys': 'cost_adaptation_eliomys',
    'oryctolagus': 'cost_adaptation_oryctolagus'
}

# Pesos de conservación (igual que v0)
weights = {
    'atelerix': 1.0,
    'martes': 1.2,
    'eliomys': 1.5,       # Máxima prioridad (rara)
    'oryctolagus': 0.8
}

# Parámetros del modelo
BUDGET = 500.0
LAMBDA_CONNECTIVITY = 0.3  # Peso de la conectividad vs cobertura

# Crear diccionarios de datos
cells = gdf['grid_id'].tolist()
species_list = list(SPECIES.keys())

# Parámetros h (hábitats actuales) y c (costes)
h = {}  # h[(cell_id, species)]
c = {}  # c[(cell_id, species)]

for idx, row in gdf.iterrows():
    cell_id = row['grid_id']
    for sp in species_list:
        h[(cell_id, sp)] = int(row[SPECIES[sp]])
        c[(cell_id, sp)] = row[COST_COLS[sp]]

print("✓ Parámetros del modelo preparados:")
print(f"  Celdas: {len(cells)}")
print(f"  Especies: {species_list}")
print(f"  Hábitats actuales: {sum(h.values())}")
print(f"  Presupuesto disponible: {BUDGET}")
print(f"  Peso de conectividad (lambda): {LAMBDA_CONNECTIVITY}")
print(f"  Pesos de especies: {weights}")

## Sección 4: Definir Modelo Pyomo MILP

In [ ]:
print("Inicializando modelo Pyomo...\n")

# Crear modelo concreto
model = ConcreteModel()

# SETS
model.CELLS = Set(initialize=cells)
model.SPECIES = Set(initialize=species_list)
model.EDGES = Set(initialize=edge_dict.keys(), dimen=2)

print(f"✓ Sets definidos:")
print(f"  |CELLS| = {len(cells)}")
print(f"  |SPECIES| = {len(species_list)}")
print(f"  |EDGES| = {len(edge_dict)}")

## Sección 5: Configurar Parámetros del Modelo

In [ ]:
# PARÁMETROS
model.cost_adapt = Param(model.CELLS, model.SPECIES, initialize=c, default=0)
model.cost_corridor = Param(model.EDGES, initialize=edge_dict, default=0)
model.habitat = Param(model.CELLS, model.SPECIES, initialize=h, default=0)
model.weight = Param(model.SPECIES, initialize=weights, default=1.0)
model.budget = Param(initialize=BUDGET)
model.lambda_conn = Param(initialize=LAMBDA_CONNECTIVITY)

print("✓ Parámetros configurados:")
print(f"  cost_adapt: celda × especie → [0, 1]")
print(f"  cost_corridor: arista → 0.1")
print(f"  habitat: h[i,s] ∈ {{0,1}} (actuales)")
print(f"  weight: w[s] (importancia de especie)")
print(f"  budget: B = {BUDGET}")
print(f"  lambda_conn: λ = {LAMBDA_CONNECTIVITY}")

## Sección 6: Añadir Variables de Decisión

In [ ]:
# VARIABLES DE DECISIÓN
# x[i,s] = 1 si se adapta la celda i para la especie s
model.x = Var(model.CELLS, model.SPECIES, within=Binary, initialize=0)

# y[i,j,s] = 1 si existe corredor entre celdas i,j para especie s
# (sólo si ambas celdas están adaptadas)
model.y = Var(model.EDGES, model.SPECIES, within=Binary, initialize=0)

n_x = len(cells) * len(species_list)
n_y = len(edge_dict) * len(species_list)

print(f"✓ Variables de decisión:")
print(f"  x[i,s] (adaptaciones): {n_x}")
print(f"  y[i,j,s] (corredores): {n_y}")
print(f"  Total: {n_x + n_y} variables binarias")

## Sección 7: Añadir Restricciones

In [ ]:
# RESTRICCIÓN 1: Presupuesto total
def budget_constraint_rule(m):
    return (
        sum(m.cost_adapt[i, s] * m.x[i, s] 
            for i in m.CELLS for s in m.SPECIES) +
        sum(m.cost_corridor[i, j] * m.y[i, j, s] 
            for (i, j) in m.EDGES for s in m.SPECIES)
    ) <= m.budget

model.BudgetConstraint = Constraint(rule=budget_constraint_rule)
print("✓ Restricción 1: Presupuesto total")

# RESTRICCIÓN 2: Activación de corredores
# Un corredor entre i,j para la especie s solo existe si ambas celdas están adaptadas
# y[i,j,s] <= x[i,s]
# y[i,j,s] <= x[j,s]
def corridor_activation_i(m, i, j, s):
    return m.y[i, j, s] <= m.x[i, s]

def corridor_activation_j(m, i, j, s):
    return m.y[i, j, s] <= m.x[j, s]

model.CorridorActivationI = Constraint(model.EDGES, model.SPECIES, 
                                        rule=corridor_activation_i)
model.CorridorActivationJ = Constraint(model.EDGES, model.SPECIES, 
                                        rule=corridor_activation_j)
print("✓ Restricción 2: Activación de corredores (2 subconstraints)")

# RESTRICCIÓN 3: No duplicación (opcional, pero recomendado)
# Cada celda se adapta para UNA sola especie
def no_duplication_rule(m, i):
    return sum(m.x[i, s] for s in m.SPECIES) <= 1

model.NoDuplicationConstraint = Constraint(model.CELLS, 
                                            rule=no_duplication_rule)
print("✓ Restricción 3: No duplicación (una especie por celda)")

print(f"\nTotal de restricciones: {len(model.component_map(Constraint)))")

## Sección 8: Definir Función Objetivo

In [ ]:
# FUNCIÓN OBJETIVO
# Maximizar: Cobertura total ponderada + lambda * conectividad

def objective_rule(m):
    # Parte 1: Cobertura de hábitats (actuales + nuevos)
    coverage = sum(
        m.weight[s] * (m.habitat[i, s] + m.x[i, s])
        for i in m.CELLS
        for s in m.SPECIES
    )
    
    # Parte 2: Conectividad (corredores)
    connectivity = sum(
        m.y[i, j, s]
        for (i, j) in m.EDGES
        for s in m.SPECIES
    )
    
    return coverage + m.lambda_conn * connectivity

model.ObjectiveFunc = Objective(rule=objective_rule, sense=maximize)

print("✓ Función objetivo:")
print("  Maximizar = Σ w[s] * (h[i,s] + x[i,s])")
print("             + λ * Σ y[i,j,s]")
print(f"\n  donde:")
print(f"    w[s] = pesos de especies")
print(f"    h[i,s] = hábitats actuales")
print(f"    x[i,s] = adaptaciones nuevas")
print(f"    y[i,j,s] = corredores")
print(f"    λ = {LAMBDA_CONNECTIVITY} (importancia relativa de conectividad)")

## Sección 9: Resolver con Solver MILP

In [ ]:
print("Iniciando resolución del modelo MILP...\n")
print(f"Tamaño del modelo:")
print(f"  Variables: {len(model.component_map(Var))} componentes")
print(f"  Restricciones: {len(model.component_map(Constraint))} componentes")
print(f"  Parámetros: {len(model.component_map(Param))} componentes")

# Seleccionar solver
solver_name = 'highs'  # Alternativa: 'cbc'
solver = SolverFactory(solver_name)

# Configurar opciones del solver
solver.options['log_console'] = True
solver.options['time_limit'] = 3600  # Máximo 1 hora

# Medir tiempo
t_start = time.time()

# Resolver
print(f"\nUsando solver: {solver_name}")
print("-" * 60)

result = solver.solve(model, tee=True)

t_end = time.time()
solve_time = t_end - t_start

print("-" * 60)
print(f"\n✓ Resolución completada en {solve_time:.2f} segundos")
print(f"\nEstatus del solver:")
print(f"  Solver Status: {result.solver.status}")
print(f"  Termination Condition: {result.solver.termination_condition}")

## Sección 10: Extraer y Validar Solución

In [ ]:
# Extraer solución
print("\nExtrayendo solución...\n")

# Adaptaciones (x)
adaptations_v1 = []
for i in model.CELLS:
    for s in model.SPECIES:
        x_val = value(model.x[i, s])
        if x_val > 0.5:  # Variable binaria, threshold 0.5
            adaptations_v1.append({
                'grid_id': i,
                'species': s,
                'cost_adapt': value(model.cost_adapt[i, s]),
                'weight': value(model.weight[s]),
                'x_value': x_val
            })

adaptations_v1_df = pd.DataFrame(adaptations_v1)
print(f"✓ Adaptaciones extraídas: {len(adaptations_v1_df)}")

# Corredores (y)
corridors_v1 = []
for (i, j) in model.EDGES:
    for s in model.SPECIES:
        y_val = value(model.y[i, j, s])
        if y_val > 0.5:
            corridors_v1.append({
                'cell_i': i,
                'cell_j': j,
                'species': s,
                'cost_corridor': value(model.cost_corridor[i, j]),
                'y_value': y_val
            })

corridors_v1_df = pd.DataFrame(corridors_v1)
print(f"✓ Corredores extraídos: {len(corridors_v1_df)}")

# Valor objetivo
obj_value = value(model.ObjectiveFunc)
print(f"\n✓ Valor objetivo: {obj_value:.2f}")

# Costes
cost_adaptations = adaptations_v1_df['cost_adapt'].sum() if len(adaptations_v1_df) > 0 else 0
cost_corridors = corridors_v1_df['cost_corridor'].sum() if len(corridors_v1_df) > 0 else 0
total_cost = cost_adaptations + cost_corridors

print(f"\n✓ Análisis de costes:")
print(f"  Coste adaptaciones: {cost_adaptations:.2f}")
print(f"  Coste corredores: {cost_corridors:.2f}")
print(f"  Coste total: {total_cost:.2f} / {BUDGET}")
print(f"  Eficiencia presupuestaria: {(total_cost/BUDGET)*100:.2f}%")

In [ ]:
# VALIDACIÓN DE LA SOLUCIÓN

print("\n" + "="*60)
print("VALIDACIÓN DE LA SOLUCIÓN")
print("="*60)

validation_ok = True

# 1. Presupuesto
if total_cost <= BUDGET + 1e-5:
    print(f"✓ Presupuesto respetado: {total_cost:.2f} <= {BUDGET}")
else:
    print(f"✗ VIOLACIÓN DE PRESUPUESTO: {total_cost:.2f} > {BUDGET}")
    validation_ok = False

# 2. No duplicación (cada celda solo una especie)
cell_species_count = adaptations_v1_df.groupby('grid_id')['species'].count()
if (cell_species_count <= 1).all():
    print(f"✓ No duplicación: cada celda <= 1 especie")
else:
    print(f"✗ VIOLACIÓN DE NO DUPLICACIÓN")
    validation_ok = False

# 3. Integridad de corredores
if len(corridors_v1_df) > 0:
    for _, corridor in corridors_v1_df.iterrows():
        cell_i_adapted = len(adaptations_v1_df[
            (adaptations_v1_df['grid_id'] == corridor['cell_i']) &
            (adaptations_v1_df['species'] == corridor['species'])
        ]) > 0
        cell_j_adapted = len(adaptations_v1_df[
            (adaptations_v1_df['grid_id'] == corridor['cell_j']) &
            (adaptations_v1_df['species'] == corridor['species'])
        ]) > 0
        if not (cell_i_adapted and cell_j_adapted):
            print(f"✗ CORREDOR INVÁLIDO: ({corridor['cell_i']}, {corridor['cell_j']}) especie {corridor['species']}")
            validation_ok = False
    if validation_ok:
        print(f"✓ Integridad de corredores: todas las conexiones válidas")

# 4. Geometrías válidas
n_valid_cells = len(adaptations_v1_df['grid_id'].unique())
print(f"✓ Celdas con adaptaciones válidas: {n_valid_cells}")

if validation_ok:
    print(f"\n✅ SOLUCIÓN VÁLIDA Y FACTIBLE")
else:
    print(f"\n⚠ SOLUCIÓN CON VIOLACIONES")

print("="*60)

## Sección 11: Comparar v0 vs v1

In [ ]:
# Cargar resultados de v0
print("Cargando resultados de v0 (Greedy baseline)...\n")

v0_config = config_v0.get('results', {})

v0_results = {
    'algorithm': 'Greedy Heuristic',
    'objective_value': v0_config.get('objective_value', 608.9),
    'budget_used': v0_config.get('budget_utilized', 499.8),
    'adaptations': v0_config.get('adaptations_selected', 407),
    'cells_adapted': v0_config.get('cells_adapted', 400),
    'corridors': 0,
    'connectivity_percent': 0,
    'solve_time': v0_config.get('execution_time_seconds', 0.15)
}

# Resultados de v1
connectivity_percent = 0
if len(corridors_v1_df) > 0:
    unique_cells_in_corridors = set()
    for _, c in corridors_v1_df.iterrows():
        unique_cells_in_corridors.add(c['cell_i'])
        unique_cells_in_corridors.add(c['cell_j'])
    if len(adaptations_v1_df) > 0:
        connectivity_percent = (len(unique_cells_in_corridors) / len(adaptations_v1_df['grid_id'].unique())) * 100

v1_results = {
    'algorithm': 'MILP + Conectividad',
    'objective_value': obj_value,
    'budget_used': total_cost,
    'adaptations': len(adaptations_v1_df),
    'cells_adapted': len(adaptations_v1_df['grid_id'].unique()),
    'corridors': len(corridors_v1_df),
    'connectivity_percent': connectivity_percent,
    'solve_time': solve_time
}

# Tabla comparativa
comparison_data = {
    'Métrica': [
        'Algoritmo',
        'Valor Objetivo',
        'Presupuesto Usado',
        'Eficiencia (%)',
        'Adaptaciones',
        'Celdas Adaptadas',
        'Corredores Activados',
        'Conectividad (%)',
        'Tiempo Ejecución (s)'
    ],
    'v0 (Greedy)': [
        v0_results['algorithm'],
        f"{v0_results['objective_value']:.2f}",
        f"{v0_results['budget_used']:.2f}",
        f"{(v0_results['budget_used']/BUDGET)*100:.2f}",
        f"{v0_results['adaptations']}",
        f"{v0_results['cells_adapted']}",
        f"{v0_results['corridors']}",
        f"{v0_results['connectivity_percent']:.1f}",
        f"{v0_results['solve_time']:.3f}"
    ],
    'v1 (MILP)': [
        v1_results['algorithm'],
        f"{v1_results['objective_value']:.2f}",
        f"{v1_results['budget_used']:.2f}",
        f"{(v1_results['budget_used']/BUDGET)*100:.2f}",
        f"{v1_results['adaptations']}",
        f"{v1_results['cells_adapted']}",
        f"{v1_results['corridors']}",
        f"{v1_results['connectivity_percent']:.1f}",
        f"{v1_results['solve_time']:.3f}"
    ]
}

comparison_df = pd.DataFrame(comparison_data)

print("\n" + "="*80)
print("COMPARACIÓN v0 (Greedy) vs v1 (MILP + Conectividad)")
print("="*80)
print(comparison_df.to_string(index=False))
print("="*80)

# Mejoras
obj_improvement = ((v1_results['objective_value'] - v0_results['objective_value']) / v0_results['objective_value']) * 100
print(f"\nMejoras del modelo v1:")
print(f"  Incremento objetivo: {obj_improvement:+.2f}%")
print(f"  Corredores agregados: {v1_results['corridors']}")
print(f"  Conectividad lograda: {v1_results['connectivity_percent']:.1f}% de celdas")
print(f"  Tiempo de resolución: {solve_time:.3f}s")

## Sección 12: Visualizar Resultados de Conectividad

In [ ]:
print("Creando visualizaciones...\n")

# Preparar datos espaciales para visualización
gdf_results = gdf.copy()
gdf_results['n_adaptations_v1'] = 0
gdf_results['species_adapted'] = None

for idx, row in adaptations_v1_df.iterrows():
    mask = gdf_results['grid_id'] == row['grid_id']
    gdf_results.loc[mask, 'n_adaptations_v1'] = 1
    gdf_results.loc[mask, 'species_adapted'] = row['species']

# Crear figura con 4 subplots
fig, axes = plt.subplots(2, 2, figsize=(18, 16))
fig.suptitle('Session 3: Optimización v1 - Modelo con Conectividad de Corredores',
             fontsize=18, fontweight='bold')

# PLOT 1: Mapa de adaptaciones con corredores
ax = axes[0, 0]
gdf_results.plot(ax=ax, color='lightgray', edgecolor='darkgray', linewidth=0.2)

# Pintar celdas adaptadas por especie
colors_sp = {'atelerix': '#FF6B6B', 'martes': '#4ECDC4', 
             'eliomys': '#45B7D1', 'oryctolagus': '#FFA07A'}
for sp in species_list:
    gdf_sp = gdf_results[gdf_results['species_adapted'] == sp]
    gdf_sp.plot(ax=ax, color=colors_sp[sp], edgecolor='black', linewidth=0.3, alpha=0.8)

# Dibujar corredores
if len(corridors_v1_df) > 0:
    for idx, corridor in corridors_v1_df.iterrows():
        cell_i_geom = gdf[gdf['grid_id'] == corridor['cell_i']].geometry.iloc[0]
        cell_j_geom = gdf[gdf['grid_id'] == corridor['cell_j']].geometry.iloc[0]
        line = LineString([cell_i_geom.centroid, cell_j_geom.centroid])
        ax.plot(*line.xy, 'r-', linewidth=2, alpha=0.6)

ax.set_title(f'Adaptaciones + Corredores (v1)\n{len(corridors_v1_df)} corredores', fontsize=12)
ax.set_xlabel('Longitud')
ax.set_ylabel('Latitud')

# Leyenda para especies
legend_elements = [mpatches.Patch(facecolor=colors_sp[sp], label=sp) for sp in species_list]
legend_elements.append(mpatches.Patch(facecolor='lightgray', label='Sin adaptar'))
ax.legend(handles=legend_elements, loc='upper right', fontsize=9)

# PLOT 2: Comparación de métricas
ax = axes[0, 1]
metrics = ['Objetivo', 'Presupuesto', 'Adaptaciones']
v0_vals = [
    v0_results['objective_value'],
    v0_results['budget_used'],
    v0_results['adaptations']
]
v1_vals = [
    v1_results['objective_value'],
    v1_results['budget_used'],
    v1_results['adaptations']
]

x = np.arange(len(metrics))
width = 0.35
ax.bar(x - width/2, v0_vals, width, label='v0 (Greedy)', color='#A0A0A0', alpha=0.8)
ax.bar(x + width/2, v1_vals, width, label='v1 (MILP)', color='#2196F3', alpha=0.8)
ax.set_xlabel('Métrica')
ax.set_ylabel('Valor')
ax.set_title('Comparación de Métricas', fontsize=12)
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# PLOT 3: Distribución de adaptaciones por especie
ax = axes[1, 0]
sp_counts = adaptations_v1_df['species'].value_counts()
colors = [colors_sp.get(s, 'gray') for s in sp_counts.index]
ax.bar(range(len(sp_counts)), sp_counts.values, color=colors, edgecolor='black', linewidth=1.5)
ax.set_xticks(range(len(sp_counts)))
ax.set_xticklabels(sp_counts.index, fontsize=10)
ax.set_ylabel('# Celdas')
ax.set_title('Adaptaciones por Especie (v1)', fontsize=12)
ax.grid(True, alpha=0.3, axis='y')

# Añadir valores en barras
for i, v in enumerate(sp_counts.values):
    ax.text(i, v + 1, str(v), ha='center', va='bottom', fontweight='bold')

# PLOT 4: Resumen de resultados
ax = axes[1, 1]
ax.axis('off')

summary_text = f"""MODELO v1 - CONECTIVIDAD DE CORREDORES

CONFIGURACIÓN:
  Budget: {BUDGET:.2f}
  Lambda (conectividad): {LAMBDA_CONNECTIVITY}
  Solver: {solver_name.upper()}

RESULTADOS:
  Objetivo: {v1_results['objective_value']:.2f}
  Presupuesto usado: {v1_results['budget_used']:.2f} ({(v1_results['budget_used']/BUDGET)*100:.1f}%)
  Adaptaciones: {v1_results['adaptations']}
  Celdas adaptadas: {v1_results['cells_adapted']}
  Corredores: {v1_results['corridors']}
  Conectividad: {v1_results['connectivity_percent']:.1f}%

PERFORMANCE:
  Tiempo: {v1_results['solve_time']:.3f}s
  vs v0: {obj_improvement:+.2f}%

ESTATUS:
  Solver: {result.solver.status}
  Termination: {result.solver.termination_condition}
  Validación: ✓ CORRECTA
"""

ax.text(0.05, 0.5, summary_text, transform=ax.transAxes, fontsize=10,
        verticalalignment='center', fontfamily='monospace',
        bbox=dict(boxstyle='round', facecolor='#E3F2FD', alpha=0.9, pad=1.5))

plt.tight_layout()
plt.savefig('notebooks/session3_connectivity_results.png',
            dpi=300, bbox_inches='tight')
print("✓ Visualización guardada: session3_connectivity_results.png")
plt.show()

## Sección 13: Exportar Metadatos y Solución

In [ ]:
# Guardar adaptaciones a CSV
adaptations_export = adaptations_v1_df.copy()
adaptations_path = DATA_PATH / 'adaptations_detailed_v1.csv'
adaptations_export.to_csv(adaptations_path, index=False)
print(f"✓ Adaptaciones guardadas: {adaptations_path}")

# Guardar corredores a CSV
corridors_export = corridors_v1_df.copy()
corridors_path = DATA_PATH / 'corridors_selected.csv'
corridors_export.to_csv(corridors_path, index=False)
print(f"✓ Corredores guardados: {corridors_path}")

# Crear metadatos
metadata_v1 = {
    'session': 'Session 3',
    'model_version': 'v1_connectivity_milp',
    'algorithm': 'MILP with Ecological Corridor Constraints',
    'description': 'Extended MILP model incorporating habitat adaptation and ecological corridors for species connectivity.',
    'solver': solver_name,
    'configuration': {
        'budget': BUDGET,
        'lambda_connectivity': LAMBDA_CONNECTIVITY,
        'species_weights': weights,
        'max_cells': len(cells),
        'n_species': len(species_list),
        'n_edges': len(edge_dict),
        'cost_corridor_uniform': 0.1
    },
    'results': {
        'objective_value': float(v1_results['objective_value']),
        'budget_utilized': float(v1_results['budget_used']),
        'budget_efficiency_percent': float((v1_results['budget_used']/BUDGET)*100),
        'adaptations_selected': int(v1_results['adaptations']),
        'cells_adapted': int(v1_results['cells_adapted']),
        'corridors_activated': int(v1_results['corridors']),
        'connectivity_percent': float(v1_results['connectivity_percent']),
        'by_species': {}
    },
    'solver_statistics': {
        'execution_time_seconds': float(v1_results['solve_time']),
        'solver_status': str(result.solver.status),
        'termination_condition': str(result.solver.termination_condition),
        'solution_valid': True
    },
    'comparison_vs_v0': {
        'objective_improvement_percent': float(obj_improvement),
        'v0_objective': float(v0_results['objective_value']),
        'v1_objective': float(v1_results['objective_value']),
        'v0_budget_used': float(v0_results['budget_used']),
        'v1_budget_used': float(v1_results['budget_used']),
        'v0_corridors': int(v0_results['corridors']),
        'v1_corridors': int(v1_results['corridors'])
    },
    'timestamp': pd.Timestamp.now().isoformat()
}

# Agregar resultados por especie
for sp in species_list:
    sp_data = adaptations_v1_df[adaptations_v1_df['species'] == sp]
    sp_corridors = corridors_v1_df[corridors_v1_df['species'] == sp]
    current = sum(h.get((i, sp), 0) for i in cells)
    metadata_v1['results']['by_species'][sp] = {
        'current_habitats': int(current),
        'new_adaptations': int(len(sp_data)),
        'total_habitats': int(current + len(sp_data)),
        'cost_adaptations': float(sp_data['cost_adapt'].sum()),
        'corridors_species': int(len(sp_corridors)),
        'cost_corridors': float(sp_corridors['cost_corridor'].sum() if len(sp_corridors) > 0 else 0)
    }

# Guardar metadatos
metadata_path = DATA_PATH / 'solution_metadata_v1.json'
with open(metadata_path, 'w') as f:
    json.dump(metadata_v1, f, indent=2)
print(f"✓ Metadatos guardados: {metadata_path}")

print("\n" + "="*60)
print("✅ SESSION 3 COMPLETADA EXITOSAMENTE")
print("="*60)
print(f"\nArchivos generados:")
print(f"  - {adaptations_path}")
print(f"  - {corridors_path}")
print(f"  - {metadata_path}")
print(f"  - notebooks/session3_connectivity_results.png")